Import libraries

In [4]:
from tqdm import tqdm 
from utils import load_pickle

Get the data

In [5]:
data_path = "/data/rbg/users/klingmin/projects/MS_processing/data/nist2023/nist2023_w_mol_info_w_CF_predictions.pkl" 
data = load_pickle(data_path)

Get accuracy of subformula annotation

In [8]:
mistakes = [] 
total_acc, total = 0, 0 

for r in tqdm(data):

    peaks = r["peaks"]
    count, correct = 0, 0 

    for p in peaks:
        
        p = p["comment"]
        if type(p) == str: continue
        if p["f"] == "": continue 
        f, f_pred = p["f"], p["f_pred"]
        
        count += 1 
        correct += int(f==f_pred)

        if f != f_pred:
            mistakes.append((f, f_pred))
    
    if count == 0: continue 

    acc = correct / count * 100 
    total_acc += acc 
    total += 1

  0%|          | 4140/1699035 [01:13<00:40, 41376.96it/s]

KeyboardInterrupt: 

Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt: 
100%|██████████| 1699035/1699035 [02:05<00:00, 13516.78it/s]

subformula accuracy: 8225.462 %


In [9]:
avg_acc = round(total_acc / total, 3)
print(f"subformula accuracy: {avg_acc} %")

subformula accuracy: 82.255 %


Get the common mistakes of subformula annotation

In [62]:
mistakes_due_to_isotopes = []
mistakes_due_to_charge = [] 
mistakes_due_to_hydrogen_rearrangements = [] 
mistakes_due_to_nitrogen_rearrangements = [] 
mistakes_due_to_unmatched = [] 
mistakes_due_to_mismatched = [] 

i = 0
for m in mistakes:

    if "+i" in m[0] or "+2i" in m[0] or "+3i" in m[0] or "-i" in m[0] or "-2i" in m[0] or "-3i" in m[0]:
        mistakes_due_to_isotopes.append(m)
    
    elif "+H2O" in m[0]:
        mistakes_due_to_hydrogen_rearrangements.append(m)
    
    elif "+N2" in m[0]:
        mistakes_due_to_nitrogen_rearrangements.append(m)
    
    elif "^2" in m[0]:
        mistakes_due_to_charge.append(m)

    elif m[1] == "":
        mistakes_due_to_unmatched.append(m)

    else:
        mistakes_due_to_mismatched.append(m)


percentage_mistakes_due_to_isotopes = round(len(mistakes_due_to_isotopes) / len(mistakes) * 100, 3)
percentage_mistakes_due_to_charge = round(len(mistakes_due_to_charge) / len(mistakes) * 100, 3)
percentage_mistakes_due_to_hydrogen_rearrangements = round(len(mistakes_due_to_hydrogen_rearrangements) / len(mistakes) * 100, 3)
percentage_mistakes_due_to_nitrogen_rearrangements = round(len(mistakes_due_to_nitrogen_rearrangements) / len(mistakes) * 100, 3)
percentage_mistakes_due_to_unmatched = round(len(mistakes_due_to_unmatched) / len(mistakes) * 100, 3)
percentage_mistakes_due_to_mismatched = round(len(mistakes_due_to_mismatched) / len(mistakes) * 100, 3)

print(f"Mistakes due to isotopes: {percentage_mistakes_due_to_isotopes}")
print(f"Mistakes due to charge: {percentage_mistakes_due_to_charge}")
print(f"Mistakes due to hydrogen: {percentage_mistakes_due_to_hydrogen_rearrangements}")
print(f"Mistakes due to nitrogen: {percentage_mistakes_due_to_nitrogen_rearrangements}")
print(f"Mistakes due to unmatch: {percentage_mistakes_due_to_unmatched}")
print(f"Mistakes due to mistmatch: {percentage_mistakes_due_to_mismatched}")

Mistakes due to isotopes: 37.806
Mistakes due to charge: 3.64
Mistakes due to hydrogen: 5.622
Mistakes due to nitrogen: 5.288
Mistakes due to unmatch: 4.424
Mistakes due to mistmatch: 43.219
